In [44]:
%load_ext autoreload
%autoreload 2

from datetime import datetime, timedelta

import pandas as pd

from short_tracker.data import (
    query_all_sec_metadata, query_uk_si_disclosures,
    SHORT_URL_UK, DATE_COL, FUND_COL, ISIN_COL, SHORT_POS_COL,
    SHARE_ISSUER_COL, UK_DISCL_THRESHOLD,
)
from short_tracker.processing import (
    check_cur_hist_discl_overlap, remove_dupl_shorts, ffill_discl_data
)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
discl_data, rept_date = query_uk_si_disclosures(SHORT_URL_UK) # exp_upd_time
rept_date

datetime.date(2022, 12, 23)

In [25]:
cur_discl = discl_data['current']
hist_discl = discl_data['historic']
all_isins = pd.concat([cur_discl[ISIN_COL], hist_discl[ISIN_COL]]).unique()

assert not check_cur_hist_discl_overlap(cur_discl, hist_discl)

hist_discl = remove_dupl_shorts(hist_discl)

exp_max_discl_date = rept_date - timedelta(days=1)
max_discl_date = cur_discl[DATE_COL].max().date()

assert max_discl_date <= exp_max_discl_date
max_discl_date

Found 3 duplicated rows:                   Position Holder       Name of Share Issuer          ISIN  Net Short Position (%) Position Date
23714  Bercheva Opportunities Ltd                  EQTEC PLC  IE00BH3XCL94                    2.19    2018-03-28
46086      Jane Street Group, LLC  MICRO FOCUS INTERNATIONAL  GB00BJ1F4N75                    0.00    2017-09-01
57940                  Roble S.L.     QUINDELL PORTFOLIO PLC  GB00BMTS9H89                    0.61    2014-02-28
Assuming the max disclosure is correct...


datetime.date(2022, 12, 22)

In [41]:
# guaranteed these funds are
cur_discl_ffill = ffill_discl_data(cur_discl, exp_max_discl_date, 0)

In [56]:
cur_discl_ind = cur_discl.set_index([ISIN_COL, FUND_COL]).index

# disclosures previously opened by a fund that are now still open
# need to ffill these up to the current disclosures
# FIXME: this logic doesn't work, see below
cont_discl_ind = hist_discl.set_index([ISIN_COL, FUND_COL]).index.isin(cur_discl_ind)
cont_hist_discl = hist_discl.loc[cont_discl_ind]

cont_discl = pd.concat([cont_hist_discl, cur_discl])

cont_discl_ffill = ffill_discl_data(cont_discl, exp_max_discl_date, UK_DISCL_THRESHOLD)

# historical shorts not currently marked as open
closed_discl = hist_discl.loc[~cont_discl_ind]

In [76]:
max_closed_idx = closed_discl.groupby([FUND_COL, ISIN_COL])[[DATE_COL]].max().set_index(DATE_COL, append=True).index
latest_closed_shorts = closed_discl[closed_discl.set_index([FUND_COL, ISIN_COL, DATE_COL]).index.isin(max_closed_idx)]

latest_closed_shorts[latest_closed_shorts[SHORT_POS_COL]>=UK_DISCL_THRESHOLD].sort_values(by=DATE_COL)

# ... so our ffill logic doesn't work since some shorts end > the threshold.

,Position Holder,Name of Share Issuer,ISIN,Position Date,Net Short Position (%)
7114,Anthion Management LLC,VALIANT PETROLEUM PLC,GB00B2NJD643,2013-03-07,1.90
68685,QUANTATATIVE LTD,RENOVO GROUP PLC,GB00B081NX89,2013-08-23,0.69
3531,"AQR Capital Management, LLC",EURASIAN NATURAL RESOURCES,GB00B29BCK10,2013-09-19,0.66
8169,"Bain Capital Public Equity Management, LLC",WEIR GROUP PLC/THE,GB0009465807,2013-12-11,0.96
65782,Oxford Asset Management,F&C ASSET MANAGEMENT,GB0004658141,2014-03-20,0.68
65631,Oxford Asset Management,AFRICAN MINERALS LTD,BMG0114P1005,2014-06-30,0.70
76563,Wolverine Asset Management LLC,LONDON MINING PLC,GB00B1VZK334,2014-08-19,0.52
37047,GSA Capital Partners LLP,AFRICAN MINERALS LTD,BMG0114P1005,2014-09-11,0.54
65903,Oxford Asset Management,LONDON MINING PLC,GB00B1VZK334,2014-09-23,0.99
65493,Oceanwood Capital Management LLP,AFRICAN MINERALS LTD,BMG0114P1005,2014-10-08,0.60


In [68]:
pivot_discl = lambda df: df.pivot(index=DATE_COL, columns=FUND_COL, values=SHORT_POS_COL)

hist_discl_dict = {isin: pivot_discl(df) for isin, df in hist_discl.groupby(ISIN_COL)}

In [259]:
sec_metadata, err_isins = query_all_sec_metadata(all_isins, "LN", "ID_ISIN")
exch_codes = {k: [x['exchCode'] for x in v] for k, v in d.items()}
print(len(err_isins))
pd.DataFrame(sec_metadata)

No data for id GB0006834344, error: None
No data for id IM00B8BV8G91, error: None
No data for id JP3325200008, error: None
No data for id GB00BKDM7X41, error: None
No data for id GB00BVFD7Q58, error: None
No data for id GB00B06GM726, error: None
No data for id GB00B09LQS34, error: None
No data for id GB00BMSKPJ95, error: None
No data for id GB00B6774699, error: None
No data for id GB00BLP5YB54, error: None
No data for id GB0000031285, error: None
No data for id GB00BMCL8X32, error: None
No data for id GB00B61D2N63, error: None
No data for id AEDFXA1EN018, error: None
No data for id GB00B0672758, error: None
No data for id IM00B39HQT38, error: None
No data for id BMG0114P1005, error: None
No data for id GB00B2QMX606, error: None
No data for id GB00BK1PTB77, error: None
No data for id GB00BQQMCJ47, error: None
No data for id GB00BQ1XTV39, error: None
No data for id GB00BLRLH124, error: None
No data for id GB0000282623, error: None
No data for id GB0032087826, error: None
No data for id G

78


,GB00BF8Q6K64,GB00B18S7B29,GB00BNDRMJ14,GB00B1XZS820,GB00BJTNFH41,GB00BYM8GJ06,GB00B132NW22,GB0030927254,GB00BVGBWW93,GB00BN7CG237,...,GB00BVG6X439,JE00B6Y3DV84,GB00BD6RF223,GB00B2PDGW16,GB00B1KJJ408,GB00BL6C2002,GB0009692319,GB00BL9YR756,GB00BVG1CF25,GB0001570810
figi,BBG000N2C023,BBG000QWQ5W5,BBG010K53NF4,BBG000BWF7M0,BBG005ZD5X21,BBG00BWCTGG9,BBG000BQY7Z8,BBG000KL73H8,BBG000CPPKR9,BBG00LY4RF68,...,BBG0086C7Q77,BBG002ZPXXB0,BBG00CFWZ1D1,BBG000MLFFM4,BBG000BRVH05,BBG00ZVX07Y2,BBG000FZGLX5,BBG011PF19N9,BBG0082YW401,BBG000BVKQF2
name,ABRDN PLC,AFC ENERGY PLC,ALPHAWAVE IP GROUP PLC,ANGLO AMERICAN PLC,AO WORLD PLC,ASCENTIAL PLC,ASHMORE GROUP PLC,ASOS PLC,ASSURA PLC,ASTON MARTIN LAGONDA GLOBAL,...,VPC SPECIALTY LENDING INVEST,WANDISCO PLC,WATKIN JONES PLC,WH SMITH PLC,WHITBREAD PLC,WICKES GROUP PLC,WILMINGTON PLC,WISE PLC - A,SCHRODER UK PUBLIC PRIVATE,XAAR PLC
ticker,ABDN,AFC,AWE,AAL,AO/,ASCL,ASHM,ASC,AGR,AML,...,VSL,WAND,WJG,SMWH,WTB,WIX,WIL,WISE,SUPP,XAR
exchCode,LN,LN,LN,LN,LN,LN,LN,LN,LN,LN,...,LN,LN,LN,LN,LN,LN,LN,LN,LN,LN
compositeFIGI,BBG000N2BZ81,BBG000QWQ5M6,BBG010K53ND6,BBG000BWF7C1,BBG000NKL498,BBG00BWCTGF0,BBG000BQY715,BBG000KL72Y1,BBG000CPPKH0,BBG00LY4RF59,...,BBG0086C7Q68,BBG002ZPXX75,BBG00CFWZ169,BBG000MLFDZ5,BBG000BRVG61,BBG00ZVX0790,BBG000FZGKX7,BBG011PF19M0,BBG0082YW3Z5,BBG000BVKQ53
securityType,Common Stock,Common Stock,Common Stock,Common Stock,Common Stock,Common Stock,Common Stock,Common Stock,REIT,Common Stock,...,Closed-End Fund,Common Stock,Common Stock,Common Stock,Common Stock,Common Stock,Common Stock,Common Stock,Closed-End Fund,Common Stock
marketSector,Equity,Equity,Equity,Equity,Equity,Equity,Equity,Equity,Equity,Equity,...,Equity,Equity,Equity,Equity,Equity,Equity,Equity,Equity,Equity,Equity
shareClassFIGI,BBG001SGL070,BBG001SS0FJ1,BBG010K53NJ0,BBG001S5Z1X8,BBG001T56Q82,BBG00BWCTGL3,BBG001SGLYM9,BBG001SJNFF0,BBG001SFN828,BBG00LY4RF95,...,BBG0086C7QB2,BBG002ZPXXM8,BBG00CFWZ150,BBG001SN9TC0,BBG001S6NGL9,BBG00ZVX07B7,BBG001S8PH77,BBG011PF19T3,BBG0082YW438,BBG001SBB0X3
securityType2,Common Stock,Common Stock,Common Stock,Common Stock,Common Stock,Common Stock,Common Stock,Common Stock,REIT,Common Stock,...,Mutual Fund,Common Stock,Common Stock,Common Stock,Common Stock,Common Stock,Common Stock,Common Stock,Mutual Fund,Common Stock
securityDescription,ABDN,AFC,AWE,AAL,AO/,ASCL,ASHM,ASC,AGR,AML,...,VSL,WAND,WJG,SMWH,WTB,WIX,WIL,WISE,SUPP,XAR


In [152]:
hist_fund_inst_shorts = pd.Index(hist_discl[[FUND_COL, ISIN_COL]].drop_duplicates())
cur_fund_inst_shorts = pd.Index(cur_discl[[FUND_COL, ISIN_COL]]) # FIXME: check for dupes

In [153]:
existing_shorts = cur_fund_inst_shorts.isin(hist_fund_inst_shorts)

cur_fund_inst_shorts_ = pd.concat([cur_discl[existing_shorts], hist_discl]) # series of shorts leading to current short
existing_fund_inst_shorts = cur_discl[~existing_shorts] # new shorts that haven't historically changed

In [104]:
import yfinance as yf
# aapl_df = yf.download('GB0006834344')
yf.Ticker("MSFT").isin

'US5949181045'

In [25]:
tickers = mapping_df.loc['ticker'] + '.LON'
tickers

GB0030927254     ASC.LON
BMG4593F1389     HSX.LON
ES0177542018     IAG.LON
GB0000055888    DSCV.LON
GB0000667013    AVON.LON
GB0000961622     BBY.LON
Name: ticker, dtype: object